# Introduction à la programmation probabiliste avec Infer.Net

## Introduction à la Programmation Probabiliste

### Avant-propos

Les ordinateurs sont rigoureusement logiques, mais le monde réel ne l'est pas. Par exemple, supposons que vous deviez représenter ce qu'un utilisateur griffonne sur un écran tactile sous forme de mot. Les gens ne sont généralement pas très appliqués dans leur écriture, donc ce gribouillis peut correspondre à plusieurs mots possibles comme "hill", "bull" ou "hello". L'utilisateur sait ce qu'il a écrit, mais pour l'application, la valeur correcte est incertaine. Cependant, certaines valeurs sont plus probables que d'autres.

Comment représentez-vous une telle incertitude dans un programme ? Des variables conventionnelles telles que `bool` ou `int` doivent avoir des valeurs bien définies. Une solution est d'utiliser des variables aléatoires.

### Variables aléatoires

La programmation probabiliste est conçue pour gérer une telle incertitude en utilisant des variables aléatoires. Une variable aléatoire représente un ensemble ou une plage de valeurs possibles, chacune associée à une probabilité.

```csharp
// Exemple de création d'une variable aléatoire
Variable<bool> isHeads = Variable.Bernoulli(0.5);
```

### Modèles probabilistes

Un modèle probabiliste définit comment les utilisateurs transforment les mots en gribouillis. Ce modèle reconnaît que le même mot peut conduire à différents gribouillis et que des mots différents peuvent conduire à des gribouillis similaires.

### Inférence probabiliste

L'inférence probabiliste utilise une méthodologie statistique connue sous le nom d'inférence bayésienne pour raisonner rétrospectivement d'une observation jusqu'à son origine.


### Apprentissage probabiliste

Un modèle a généralement un ensemble de paramètres ajustables qui régissent son comportement. Pour adapter les paramètres au style d’écriture d’un utilisateur, un programme probabiliste peut traiter les paramètres du modèle eux-mêmes comme des variables aléatoires et apprendre les valeurs réelles des paramètres en fonction des entrées de l’utilisateur.

## Présentation d'Infer.Net

### Qu’est-ce que Infer.Net

Infer.NET est un framework permettant d'exécuter l'inférence bayésienne dans des modèles graphiques. Il fournit les algorithmes de passe-messages et les routines statistiques nécessaires à la réalisation d'inférences pour une grande variété d'applications. 

Les principales caractéristiques d'Infer.NET sont :

- **Langage de modélisation** : Supporte les variables univariées et multivariées, continues et discrètes.
- **Algorithmes d'inférence** : Inclut la propagation d’espérance (EP), la propagation des convictions, le passage de messages variationnels et l'échantillonnage de Gibbs.
- **Conçu pour l'inférence à grande échelle** : Compile les modèles dans du code source dédié.
- **Extensible par l'utilisateur** : Permet d'ajouter des distributions de probabilité, des facteurs, des opérations de message et des algorithmes d'inférence.

### Comment fonctionne Infer.NET

Infer.NET fonctionne en compilant une définition de modèle dans le code source nécessaire pour calculer un ensemble de requêtes d'inférence sur le modèle.

1. **Création de la définition de modèle** : Utilisation de l'API de modélisation.
2. **Compilation du modèle** : Le compilateur crée le code source nécessaire à l'exécution des requêtes d'inférence.
3. **Compilation du code source** : Pour créer un algorithme compilé.
4. **Exécution de l'inférence** : Utilisation du moteur d'inférence pour produire les distributions marginales demandées.

### Installation des Packages Nuget et Usings

Pour utiliser Infer.NET dans votre projet .NET Interactive, vous devez installer le package Nuget `Microsoft.ML.Probabilistic` et inclure les usings nécessaires.

In [1]:
// Pour installer Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

(Infer fait désormais parti de la bibliothèque ML.Net)

### Import des espaces de noms essentiels

In [2]:
using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;


### Un exemple simple

Voici un exemple d'utilisation d'Infer.NET pour déterminer la probabilité d'obtenir deux faces lors du lancement de deux pièces non biaisées.




In [3]:
Variable<bool> premierePiece = Variable.Bernoulli(0.5);
Variable<bool> deuxiemePiece = Variable.Bernoulli(0.5);
Variable<bool> deuxFaces = premierePiece & deuxiemePiece;

InferenceEngine engine = new InferenceEngine();
engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
Console.WriteLine("Probabilité d’avoir deux faces: " + engine.Infer(deuxFaces));

Ce court exemple contient les trois éléments clés de tout programme Infer.NET :

1. **Définition d'un modèle probabiliste** : Les variables aléatoires `premierePiece` et `deuxiemePiece` et la variable dépendante `deuxFaces`.
2. **Création d'un moteur d'inférence** : Le moteur d'inférence est créé et configuré.
3. **Exécution d'une requête d'inférence** : Le moteur est utilisé pour déduire la distribution marginale de `deuxFaces`.


## Mise en œuvre d’Infer.Net dans un exemple détaillé

### Contexte

#### Scenario

Les programmes présentés ici sont basés sur le scénario suivant :

- Plusieurs de vos collègues et vous-même rendez-vous à vélo chaque jour au travail.
- Le temps de trajet d’un cycliste varie de jour en jour et sa valeur est donc aléatoire.
- L’incertitude du temps de trajet est représentée par une distribution de probabilité qui définit le temps de trajet moyen et combien il varie.
- L’application apprendra cette distribution à partir de plusieurs temps de trajet observés et utilisera ces connaissances pour faire des prévisions sur les temps de déplacement futurs.

### Création du modèle

La première étape consiste à créer les variables aléatoires `dureeMoyenne` et `bruitTrafic` avec des distributions initiales.

In [4]:
// Définir le modèle
Variable<double> dureeMoyenne = Variable.GaussianFromMeanAndPrecision(15, 0.01);
Variable<double> bruitTrafic = Variable.GammaFromShapeAndScale(2, 0.5);

Ensuite, nous définissons les variables pour les temps de trajet observés (`dureeLundi`, `dureeMardi`, `dureeMercredi`).

In [5]:
// Définir les temps de trajet
Variable<double> dureeLundi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Variable<double> dureeMardi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Variable<double> dureeMercredi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);

### Entraînement du modèle

Nous allons maintenant observer les temps de trajet pour trois jours et entraîner le modèle.

In [6]:
// Observations des temps de trajet
dureeLundi.ObservedValue = 13;
dureeMardi.ObservedValue = 17;
dureeMercredi.ObservedValue = 16;

Pour calculer les distributions postérieures pour `dureeMoyenne` et `bruitTrafic`, nous utilisons le moteur d'inférence.


In [7]:
// Entraînement du modèle et calcul des postérieurs
InferenceEngine engine = new InferenceEngine();
engine.Compiler.CompilerChoice = CompilerChoice.Roslyn;
Gaussian moyennePosterieure = engine.Infer<Gaussian>(dureeMoyenne);
Gamma bruitPosterieur = engine.Infer<Gamma>(bruitTrafic);

Console.WriteLine($"Moyenne a posteriori: {moyennePosterieure}");
Console.WriteLine($"Bruit traffic a posteriori: {bruitPosterieur}");

### Prédiction du temps de trajet

Nous allons maintenant utiliser le modèle entraîné pour prédire le temps de trajet de demain (`dureeDemain`).

In [8]:
// Prédiction du temps de trajet de demain
Variable<double> dureeDemain = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Gaussian distribDemain = engine.Infer<Gaussian>(dureeDemain);

Console.WriteLine($"Prédiction demain: {distribDemain}, écart type: {Math.Sqrt(distribDemain.GetVariance())}");

### Calculer des probabilités basées sur les prédictions

Nous pouvons également calculer la probabilité que le trajet de demain prenne moins de 18 minutes.

In [9]:
// Calcul de la probabilité
double probMoinsDe18Mn = engine.Infer<Bernoulli>(dureeDemain < 18.0).GetProbTrue();
Console.WriteLine($"Probabilité que le trajet prenne moins de 18mn: {probMoinsDe18Mn:F2}");

### Résumé 

En résumé, nous avons défini un modèle probabiliste pour les temps de trajet d'un cycliste, observé les données pour entraîner le modèle, et utilisé ce modèle pour faire des prédictions sur les temps de trajet futurs. Le code est structuré de manière à tirer parti des capacités interactives du notebook .NET Interactive, permettant ainsi une approche progressive et claire.

## Restructuration de notre application

### Introduction

L'application `DureeCycliste1` a présenté les bases de l’apprentissage des paramètres, mais elle n'est pas facilement extensible pour des scénarios plus sophistiqués. Nous allons restructurer l'application pour encapsuler le code de modélisation dans des classes séparées et introduire l'utilisation de tableaux de variables aléatoires pour les variables observées.

### Encapsulation dans des classes séparées

Nous allons créer une base commune pour les modèles d'entraînement et de prédiction en utilisant une classe de base `CyclisteBase`, puis créer des classes spécifiques pour l'entraînement (`EntrainementCycliste`) et la prédiction (`PredictionCycliste`).

#### Classe de base

La classe `CyclisteBase` contient les éléments communs aux modèles d'entraînement et de prédiction.

In [11]:
using Microsoft.ML.Probabilistic.Algorithms;

public struct DonneesCycliste
{
    public Gaussian DistribMoyenne;
    public Gamma DistribBruitTraffic;
    public DonneesCycliste(Gaussian moyenne, Gamma precision)
    {
        DistribMoyenne = moyenne;
        DistribBruitTraffic = precision;
    }
}

public class CyclisteBase
{
    public InferenceEngine MoteurInference;
    protected Variable<double> Moyenne;
    protected Variable<double> Bruit;
    protected Variable<Gaussian> MoyenneAPriori;
    protected Variable<Gamma> BruitAPriori;

    public virtual void CreationModeleBayesien()
    {
        MoyenneAPriori = Variable.New<Gaussian>();
        BruitAPriori = Variable.New<Gamma>();
        Moyenne = Variable.Random<double, Gaussian>(MoyenneAPriori);
        Bruit = Variable.Random<double, Gamma>(BruitAPriori);

        if (MoteurInference == null)
        {
            MoteurInference = new InferenceEngine(new ExpectationPropagation());
            MoteurInference.Compiler.CompilerChoice = CompilerChoice.Roslyn;
        }
    }

    public virtual void DefinirDistributions(DonneesCycliste distribsApriori)
    {
        MoyenneAPriori.ObservedValue = distribsApriori.DistribMoyenne;
        BruitAPriori.ObservedValue = distribsApriori.DistribBruitTraffic;
    }
    
}

#### Classe d'entraînement

La classe `EntrainementCycliste` hérite de `CyclisteBase` et implémente le modèle d'entraînement.

In [12]:
public class EntrainementCycliste : CyclisteBase
{
    protected VariableArray<double> TempsDeTrajet;
    protected Variable<int> NombreDeTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        using (Variable.ForEach(indiceTrajet))
        {
            TempsDeTrajet[indiceTrajet] = Variable.GaussianFromMeanAndPrecision(Moyenne, Bruit);
        }
    }

    public DonneesCycliste CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCycliste posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian>(Moyenne);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma>(Bruit);
        return posterieurs;
    }
}

#### Classe de prédiction

La classe `PredictionCycliste` implémente le modèle de prédiction.

In [13]:
public class PredictionCycliste : CyclisteBase
{
    private Gaussian demainDistrib;
    public Variable<double> demainTemps;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        demainTemps = Variable.GaussianFromMeanAndPrecision(Moyenne, Bruit);
    }

    public Gaussian EstimerTempsDemain()
    {
        demainDistrib = MoteurInference.Infer<Gaussian>(demainTemps);
        return demainDistrib;
    }

    public Bernoulli EstimerTempsDemainInferieurA(double duree)
    {
        return MoteurInference.Infer<Bernoulli>(demainTemps < duree);
    }
}

### Utilisation du modèle

Nous allons utiliser ces classes pour entraîner le modèle et prédire le temps de déplacement de demain.

#### Entraînement

In [14]:
double[] donneesTrajets = new[] { 13, 17, 20, 25, 16, 11, 16, 14, 12.5 };
DonneesCycliste mesDistributions = new DonneesCycliste(
    Gaussian.FromMeanAndPrecision(1, 0.01),
    Gamma.FromShapeAndScale(2, 0.5));

EntrainementCycliste monEntrainement = new EntrainementCycliste();
monEntrainement.CreationModeleBayesien();
monEntrainement.DefinirDistributions(mesDistributions);
DonneesCycliste monPosterieur = monEntrainement.CalculePosterieurs(donneesTrajets);

Console.WriteLine($"Moyenne a posteriori: {monPosterieur.DistribMoyenne}");
Console.WriteLine($"Bruit traffic a posteriori: {monPosterieur.DistribBruitTraffic}");

#### Prédiction

In [15]:
PredictionCycliste maPrediction = new PredictionCycliste();
maPrediction.CreationModeleBayesien();
maPrediction.DefinirDistributions(monPosterieur);
Gaussian distribDemain = maPrediction.EstimerTempsDemain();

Console.WriteLine($"Prédiction demain: {distribDemain}, écart type: {Math.Sqrt(distribDemain.GetVariance())}");
double probMoinsDe18Mn = maPrediction.EstimerTempsDemainInferieurA(13).GetProbTrue();
Console.WriteLine($"Probabilité que le trajet prenne moins de 18mn: {probMoinsDe18Mn:F2}");

Avec cette approche, nous avons encapsulé les modèles d'entraînement et de prédiction dans des classes séparées tout en mutualisant les parties communes dans une classe de base. Cela permet de gérer efficacement plusieurs ensembles d'observations et améliore les performances en évitant la recompilation du modèle.

### Apprentissage en ligne avec Infer.NET

L'application `DureeCycliste3` implémente l'apprentissage en ligne et la gestion des événements extraordinaires en utilisant un modèle de mélange de Gaussiennes. Cette méthode permet d'apprendre progressivement les paramètres du modèle en fonction des nouvelles données, améliorant ainsi les prévisions.

### 1. Mise à jour des postérieurs

Dans l'apprentissage en ligne, nous commençons par des a priori initiaux larges, puis nous les mettons à jour avec les postérieurs calculés à partir des nouvelles données. Voici comment implémenter ce processus :

#### Code d'apprentissage en ligne

In [16]:
double[] semaineSuivante = new double[] { 18, 25, 30, 14, 11 };
maPrediction.DefinirDistributions(monPosterieur);
DonneesCycliste posterieurSemaineSuivante = monEntrainement.CalculePosterieurs(semaineSuivante);
Console.WriteLine($"moyenne postérieure semaine suivante: {posterieurSemaineSuivante.DistribMoyenne}");
Console.WriteLine($"bruit postérieur semaine suivante: {posterieurSemaineSuivante.DistribBruitTraffic}");

### 2. Nouvelles prédictions

Après avoir mis à jour les postérieurs avec les nouvelles données, nous utilisons ces postérieurs pour faire de nouvelles prédictions.

#### Code de prédiction


In [17]:
maPrediction.DefinirDistributions(posterieurSemaineSuivante);
Gaussian distribDemain = maPrediction.EstimerTempsDemain();
Console.WriteLine($"Prédiction demain: {distribDemain}");
Console.WriteLine($"Ecart type: {Math.Sqrt(distribDemain.GetVariance())}");
double probMoinsDe18Mn = maPrediction.EstimerTempsDemainInferieurA(13).GetProbTrue();
Console.WriteLine($"Probabilité que le trajet prenne moins de 18mn: {probMoinsDe18Mn:F2}");

### Modèle mixte

Pour gérer des événements extraordinaires, nous utilisons un modèle de mélange de deux Gaussiennes. Ce modèle permet de mieux capturer la variabilité des temps de trajet.

#### Classe de base pour le modèle mixte

In [18]:
public struct DonneesCyclisteMixte
    {
        public Gaussian[] DistribMoyenne;
        public Gamma[] DistribBruitTraffic;
        public Dirichlet DistribMixe;
    }

public class CyclisteBaseMixte
{
    public InferenceEngine MoteurInference;
    protected int NombreComposantes = 2;
    protected VariableArray<Gaussian> MoyennesAPriori;
    protected VariableArray<Gamma> BruitsAPriori;
    protected Variable<Dirichlet> MixeAPriori;
    protected VariableArray<double> Moyennes;
    protected VariableArray<double> Bruits;
    protected Variable<Vector> Mixe;

    public virtual void CreationModeleBayesien()
    {
        Range indiceComposants = new Range(NombreComposantes);
        MoteurInference = new InferenceEngine(new VariationalMessagePassing());
        MoteurInference.Compiler.CompilerChoice = CompilerChoice.Roslyn;
        MoyennesAPriori = Variable.Array<Gaussian>(indiceComposants);
        BruitsAPriori = Variable.Array<Gamma>(indiceComposants);
        Moyennes = Variable.Array<double>(indiceComposants);
        Bruits = Variable.Array<double>(indiceComposants);
        using (Variable.ForEach(indiceComposants))
        {
            Moyennes[indiceComposants] = Variable<double>.Random(MoyennesAPriori[indiceComposants]);
            Bruits[indiceComposants] = Variable<double>.Random(BruitsAPriori[indiceComposants]);
        }
        MixeAPriori = Variable.New<Dirichlet>();
        Mixe = Variable<Vector>.Random(MixeAPriori);
        Mixe.SetValueRange(indiceComposants);
    }

    public virtual void DefinirDistributions(DonneesCyclisteMixte distribsApriori)
    {
        MoyennesAPriori.ObservedValue = distribsApriori.DistribMoyenne;
        BruitsAPriori.ObservedValue = distribsApriori.DistribBruitTraffic;
        MixeAPriori.ObservedValue = distribsApriori.DistribMixe;
    }

    
}

#### Classe d'entraînement pour le modèle mixte

In [19]:
public class EntrainementCyclisteMixte : CyclisteBaseMixte
{
    protected Variable<int> NombreDeTrajets;
    protected VariableArray<double> TempsDeTrajet;
    protected VariableArray<int> ComposantesTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        ComposantesTrajets = Variable.Array<int>(indiceTrajet);
        using (Variable.ForEach(indiceTrajet))
        {
            ComposantesTrajets[indiceTrajet] = Variable.Discrete(Mixe);
            using (Variable.Switch(ComposantesTrajets[indiceTrajet]))
            {
                TempsDeTrajet[indiceTrajet].SetTo(Variable.GaussianFromMeanAndPrecision(Moyennes[ComposantesTrajets[indiceTrajet]], Bruits[ComposantesTrajets[indiceTrajet]]));
            }
        }
    }

    public DonneesCyclisteMixte CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCyclisteMixte posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian[]>(Moyennes);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma[]>(Bruits);
        posterieurs.DistribMixe = MoteurInference.Infer<Dirichlet>(Mixe);
        return posterieurs;
    }
}

#### Classe de prédiction pour le modèle mixte

In [20]:
public class PredictionCyclisteMixte : CyclisteBaseMixte
{
    private Gaussian demainDistrib;
    public Variable<double> demainTemps;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        Variable<int> indiceComposant = Variable.Discrete(Mixe);
        demainTemps = Variable.New<double>();
        using (Variable.Switch(indiceComposant))
        {
            demainTemps.SetTo(Variable.GaussianFromMeanAndPrecision(Moyennes[indiceComposant], Bruits[indiceComposant]));
        }
    }

    public Gaussian EstimerTempsDemain()
    {
        demainDistrib = MoteurInference.Infer<Gaussian>(demainTemps);
        return demainDistrib;
    }
}

### Utilisation du modèle mixte

#### Entraînement


In [21]:
double[] donneesTrajets = new[] { 13, 17, 20, 25, 16, 11, 16, 25, 12.5, 30 };
DonneesCyclisteMixte mesDistribsAPriori = new DonneesCyclisteMixte
{
    DistribMoyenne = new Gaussian[]
    {
        new Gaussian(15, 100), // Ordinaire
        new Gaussian(30, 100) // Extraordinaire
    },
    DistribBruitTraffic = new Gamma[]
    {
        new Gamma(2, 0.5), // O
        new Gamma(2, 0.5) // E
    },
    DistribMixe = new Dirichlet(1, 1)
};
EntrainementCyclisteMixte monEntrainement = new EntrainementCyclisteMixte();
monEntrainement.CreationModeleBayesien();
monEntrainement.DefinirDistributions(mesDistribsAPriori);
DonneesCyclisteMixte posterieur = monEntrainement.CalculePosterieurs(donneesTrajets);
Console.WriteLine($"moyenne postérieure 1: {posterieur.DistribMoyenne[0]}");
Console.WriteLine($"bruit postérieur 1: {posterieur.DistribBruitTraffic[0]}");
Console.WriteLine($"moyenne postérieure 2: {posterieur.DistribMoyenne[1]}");
Console.WriteLine($"bruit postérieur 2: {posterieur.DistribBruitTraffic[1]}");
Console.WriteLine($"Coefficients du mélange: {posterieur.DistribMixe}");

#### Prédiction

In [22]:
// 2 - Prédiction
PredictionCyclisteMixte maPrediction = new PredictionCyclisteMixte();
maPrediction.CreationModeleBayesien();
maPrediction.DefinirDistributions(posterieur);
Gaussian distribDemain = maPrediction.EstimerTempsDemain();
Console.WriteLine($"Prédiction demain: {distribDemain}");
Console.WriteLine($"Ecart type: {Math.Sqrt(distribDemain.GetVariance())}");

### Résultats

Les résultats montrent que le modèle prédit une durée de trajet moyen plus élevée et un écart type plus important en tenant compte des événements extraordinaires, ce qui rend le modèle plus réaliste et adapté aux variations réelles des trajets.

### Comparaison de Modèles

Nous disposons maintenant de deux modèles pour représenter le temps de trajet des cyclistes : l'un basé sur une seule gaussienne et l'autre sur un mélange de deux gaussiennes. Pourquoi ne pas envisager un modèle basé sur un mélange de trois gaussiennes, ou vingt ou trente gaussiennes ? Ou bien est-ce qu'une seule gaussienne suffit ? Comment choisir le meilleur modèle ?

En général, un modèle complexe avec plus de paramètres ajustables représente un ensemble de données particulier plus précisément qu'un modèle plus simple avec moins de paramètres. La question plus intéressante est de savoir si le modèle plus complexe offre une meilleure représentation qu'un modèle plus simple. Les modèles trop adaptés à un ensemble de données particulier ne seront pas utiles en pratique car ils ne s'ajusteront pas nécessairement bien aux nouvelles données.

Par exemple, lorsque vous ajustez un polynôme à un ensemble de points de données, vous pouvez toujours obtenir un ajustement exact en ajoutant suffisamment d'éléments au polynôme. Cependant, un modèle qui correspond exactement à chaque point de données fluctue généralement de manière erratique entre les points et ne correspondra donc pas bien aux nouvelles données. Ce phénomène est connu sous le nom de surapprentissage (overfitting).

La meilleure approche consiste à trouver un modèle qui ajuste raisonnablement bien les données sans être trop complexe. Comparer visuellement les modèles avec les données est subjectif et peu pratique pour des scénarios plus complexes. Ce qu'il vous faut est un critère objectif, comme le rasoir d'Occam, qui évalue quantitativement la qualité du modèle et détermine le compromis optimal entre précision et complexité. En inférence bayésienne, il existe un mécanisme robuste pour évaluer la qualité du modèle appelé preuve du modèle (evidence).

### Calculer la preuve du modèle avec Infer.NET

Avec Infer.NET, la preuve est représentée par une variable aléatoire booléenne. La procédure de base est illustrée dans l'exemple suivant.

In [84]:
Variable<bool> Evidence = Variable.Bernoulli(0.5);
using (Variable.If(Evidence))
{
    // Implémenter le modèle d'entraînement à évaluer
}
// Observer les données d'entraînement
// Interroger le moteur d'inférence pour les postérieurs du modèle
// Interroger le moteur d'inférence pour la distribution de la preuve

Infer.NET définit des modèles à deux branches en utilisant `Variable.If` et `Variable.IfNot`, qui sont l'équivalent de `if-else` en C#. La condition est une variable aléatoire booléenne. La probabilité que la condition soit vraie détermine la proportion de la branche `If` dans le mélange, et le reste du mélange est la branche `IfNot`.

Pour évaluer la preuve d'un modèle, utilisez `If/IfNot` comme illustré dans l'exemple, ce qui crée un mélange de deux modèles :

- Le modèle que vous souhaitez évaluer, représenté par la branche `If`.
- Un modèle "vide", représenté par la branche `IfNot` manquante.

La variable `Preuve` est la condition qui contrôle les proportions dans le mélange. Son a priori initial est généralement défini sur `Bernoulli(0.5)`. Pour déterminer la distribution réelle de la preuve, observez les données et calculez le postérieur de la variable `Preuve`.

### Implémentation de 

La nouvelle application utilise la preuve pour évaluer si le modèle `EntrainementCycliste` ou `EntrainementCyclisteMixte` représente le mieux les données d'entraînement de `EntrainementCyclisteMixte`.

#### Classe CyclisteAvecPreuve

La classe `CyclisteAvecPreuve` évalue la preuve pour le modèle `EntrainementCycliste`.

In [24]:
public class CyclisteAvecPreuve : EntrainementCycliste
{
    protected Variable<bool> Preuve;

    public override void CreationModeleBayesien()
    {
        Preuve = Variable.Bernoulli(0.5);
        using (Variable.If(Preuve))
        {
            base.CreationModeleBayesien();
        }
    }

    public double CalculPreuve(double[] trainingData)
    {
        double logEvidence;
        DonneesCycliste posteriors = base.CalculePosterieurs(trainingData);
        logEvidence = MoteurInference.Infer<Bernoulli>(Preuve).LogOdds;
        return logEvidence;
    }
}

#### Classe CyclisteMixedAvecPreuve

La classe `CyclisteMixedAvecPreuve` évalue la preuve pour le modèle `EntrainementCyclisteMixte`.

In [25]:
public class CyclisteMixedAvecPreuve : EntrainementCyclisteMixte
{
    protected Variable<bool> Preuve;

    public override void CreationModeleBayesien()
    {
        Preuve = Variable.Bernoulli(0.5);
        using (Variable.If(Preuve))
        {
            base.CreationModeleBayesien();
        }
    }

    public double CalculPreuve(double[] trainingData)
    {
        double logEvidence;
        DonneesCyclisteMixte posteriors = base.CalculePosterieurs(trainingData);
        logEvidence = MoteurInference.Infer<Bernoulli>(Preuve).LogOdds;
        return logEvidence;
    }
}

#### Calcul de la preuve

On calcule la preuve pour les deux modèles et on affiche les résultats.

In [26]:
double[] trainingData = new double[] { 13, 17, 16, 12, 13, 12, 14, 18, 16, 16, 27, 32 };

DonneesCycliste initPriors = new DonneesCycliste(
    Gaussian.FromMeanAndPrecision(15.0, 0.01),
    Gamma.FromShapeAndScale(2.0, 0.5));

CyclisteAvecPreuve cyclistWithEvidence = new CyclisteAvecPreuve();
cyclistWithEvidence.CreationModeleBayesien();
cyclistWithEvidence.DefinirDistributions(initPriors);
double logEvidence = cyclistWithEvidence.CalculPreuve(trainingData);
// Preuves pour le modèle CyclistMixedWithEvidence (CyclingTime3)
DonneesCyclisteMixte initPriorsMixed = new DonneesCyclisteMixte
{
    DistribMoyenne = new Gaussian[] { new Gaussian(15.0, 100), new Gaussian(30.0, 100) },
    DistribBruitTraffic = new Gamma[] { new Gamma(2.0, 0.5), new Gamma(2.0, 0.5) },
    DistribMixe = new Dirichlet(1, 1)
};
CyclisteMixedAvecPreuve cyclistMixedWithEvidence = new CyclisteMixedAvecPreuve();
cyclistMixedWithEvidence.CreationModeleBayesien();
cyclistMixedWithEvidence.DefinirDistributions(initPriorsMixed);
double logEvidenceMixed = cyclistMixedWithEvidence.CalculPreuve(trainingData);
// Affichage des résultats
Console.WriteLine($"Preuve logarithmique pour une gaussienne simple : {logEvidence}");
Console.WriteLine($"Preuve logarithmique pour un mélange de deux gaussiennes : {logEvidenceMixed}");

Ces valeurs de preuve montrent que le modèle de mélange de deux gaussiennes est supérieur au modèle basé sur une seule gaussienne pour les données d'entraînement fournies.